# DL2DWH Lab

In this lab you are asked to cover the following points:
1. Data ingestion
2. Data understanding
3. Design and build a relational DWH
4. Query the DWH with Tableau (Desktop)
5. (Bonus) Plot the provenance graph

In [1]:
!pip install awswrangler
!pip install pandas
!pip install sklearn
!pip install prov
!pip install seaborn
!pip install pydot
!pip install graphviz
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 KB 6.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 33.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 KB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 97.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 KB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 KB 3.3 MB/s eta 

In [3]:
import pandas as pd
import awswrangler as wr
import seaborn as sns
import prov

## Data ingestion

- Upload the following files in S3 (remember the naming conventions of the data lake areas!)
    - [http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-0-1.v0.0.1.csv](http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-0-1.v0.0.1.csv)
    - [http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-1-2.v0.0.1.csv](http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-1-2.v0.0.1.csv)
    - [http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-2-3.v0.0.1.csv](http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-2-3.v0.0.1.csv)
- Merge/Append them into a single Pandas DataFrame
    - Which is the underlying assumption so that `append` works?
    - See [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html)
- Save the processed dataset into S3

In [9]:
file_names = ["s3://staging-clean-sales-911/sales_fact_1997.slice-0-1.v0.0.1.csv", "s3://staging-clean-sales-911/sales_fact_1997.slice-1-2.v0.0.1.csv", "s3://staging-clean-sales-911/sales_fact_1997.slice-2-3.v0.0.1.csv",]
    
df = pd.concat(map(pd.read_csv, ["s3://landing-raw-sales-911/sales_fact_1997.slice-0-1.v0.0.1.csv", 
                                "s3://landing-raw-sales-911/sales_fact_1997.slice-1-2.v0.0.1.csv", 
                                "s3://landing-raw-sales-911/sales_fact_1997.slice-2-3.v0.0.1.csv"]), ignore_index = True)
print(df)
    

      C. City C. Country C. State  Customer Yearly Income           Category  \
0      Albany        USA       OR       117  $90K - $110K       Baking Goods   
1      Albany        USA       OR       117  $90K - $110K       Baking Goods   
2      Albany        USA       OR       117  $90K - $110K       Baking Goods   
3      Albany        USA       OR       117  $90K - $110K  Bathroom Products   
4      Albany        USA       OR       117  $90K - $110K      Beer and Wine   
...       ...        ...      ...       ...           ...                ...   
86824  Yakima        USA       WA     10226   $10K - $30K   Jams and Jellies   
86825  Yakima        USA       WA     10226   $10K - $30K               Meat   
86826  Yakima        USA       WA     10226   $10K - $30K              Pizza   
86827  Yakima        USA       WA     10226   $10K - $30K         Vegetables   
86828  Yakima        USA       WA     10226   $10K - $30K         Vegetables   

            Subcategory          Family

In [10]:
landing_bucket = "s3://landing-raw-sales-911/"
staging_bucket = "s3://staging-clean-sales-911/"

In [ ]:
# wr.s3.to_csv(df=df1, path=output_bucket + "cleaned-v1-" + file_name, index=False)

wr.s3.to_csv(df = df, path = staging_bucket + "merged-sales-fact", index = False)

## Data understanding

- What is the dataset about?
- Which are candidate levels/measures?
- Check and fix the missing values (if any)
- How many products have been sold?
    - See [https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)
- Which functional dependencies hold among the levels?
    - Given a relation R, a set of attributes X in R is said to functionally determine another set of attributes Y, also in R, (written X → Y) if, and only if, each X value is associated with precisely one Y value; R is then said to satisfy the functional dependency X → Y.
    - See [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html)
    - Hint: `df.groupby([...])` + `nunique()`
- Are there relationships between measures?
    - See [https://seaborn.pydata.org/generated/seaborn.pairplot.html](https://seaborn.pydata.org/generated/seaborn.pairplot.html)
    - See [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)

In [19]:
print(df.isnull().sum())

C. City          0
C. Country       0
C. State         0
Customer         0
Yearly Income    0
Category         0
Subcategory      0
Family           0
Product          0
S. City          0
S. Country       0
S. State         0
Store            0
Type             0
Sales Date       0
Store Cost       0
Store Sales      0
Unit Sales       0
dtype: int64


In [20]:
print(df.columns)

Index(['C. City', 'C. Country', 'C. State', 'Customer', 'Yearly Income',
       'Category', 'Subcategory', 'Family', 'Product', 'S. City', 'S. Country',
       'S. State', 'Store', 'Type', 'Sales Date', 'Store Cost', 'Store Sales',
       'Unit Sales'],
      dtype='object')


In [24]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86829 entries, 0 to 86828
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   C. City        86829 non-null  object 
 1   C. Country     86829 non-null  object 
 2   C. State       86829 non-null  object 
 3   Customer       86829 non-null  int64  
 4   Yearly Income  86829 non-null  object 
 5   Category       86829 non-null  object 
 6   Subcategory    86829 non-null  object 
 7   Family         86829 non-null  object 
 8   Product        86829 non-null  object 
 9   S. City        86829 non-null  object 
 10  S. Country     86829 non-null  object 
 11  S. State       86829 non-null  object 
 12  Store          86829 non-null  int64  
 13  Type           86829 non-null  object 
 14  Sales Date     86829 non-null  object 
 15  Store Cost     86829 non-null  float64
 16  Store Sales    86829 non-null  float64
 17  Unit Sales     86829 non-null  int64  
dtypes: flo

In [26]:
List of columns: ['C. City', 'C. Country', 'C. State', 'Customer', 'Yearly Income',
       'Category', 'Subcategory', 'Family', 'Product', 'S. City', 'S. Country',
       'S. State', 'Store', 'Type', 'Sales Date', 'Store Cost', 'Store Sales',
       'Unit Sales']
    
FD:
City -> State -> Country [to be checked]
Category -> Subcategory -> Family -> Product [to be checked]
    
DT_Customer = (C.City, C.Country, C.State, Customer, Yearly_Income)
DT_Product = (Category, Subcategory, Family, Product)
DT_Store = (S.City, S.Country, S.State, Store, Type)

FT_Sale = (Customer:DT_Customer, Product:DT_Product, Sales_Date, *Store_Cost*, *Store_Sales*, *Unit_Sales*)

*Measures*
-Level-

SyntaxError: invalid syntax (3818622918.py, line 1)

## Design and build a relational DWH

- Create the database on Amazon RDS
- Design the relational schema of the DWH (which are the functional dependencies?)
- Upload the tables into rhw DWH

In [28]:
raw_data = wr.s3.read_csv(staging_bucket + "merged-sales-fact")
raw_data

,C. City,C. Country,C. State,Customer,Yearly Income,Category,Subcategory,Family,Product,S. City,S. Country,S. State,Store,Type,Sales Date,Store Cost,Store Sales,Unit Sales
0,Albany,USA,OR,117,$90K - $110K,Baking Goods,Cooking Oil,Food,Super Corn Oil,Salem,USA,OR,13,Deluxe Supermarket,1/21/1997 12:00:00 AM,2.1177,5.43,3
1,Albany,USA,OR,117,$90K - $110K,Baking Goods,Sauces,Food,Landslide Tomato Sauce,Salem,USA,OR,13,Deluxe Supermarket,1/21/1997 12:00:00 AM,1.6500,5.00,4
2,Albany,USA,OR,117,$90K - $110K,Baking Goods,Sugar,Food,Landslide Brown Sugar,Salem,USA,OR,13,Deluxe Supermarket,1/21/1997 12:00:00 AM,5.3760,11.20,4
3,Albany,USA,OR,117,$90K - $110K,Bathroom Products,Conditioner,Non-Consumable,Consolidated Silky Smooth Hair Conditioner,Salem,USA,OR,13,Deluxe Supermarket,1/21/1997 12:00:00 AM,1.1466,2.94,2
4,Albany,USA,OR,117,$90K - $110K,Beer and Wine,Wine,Drink,Pearl Merlot Wine,Salem,USA,OR,13,Deluxe Supermarket,1/21/1997 12:00:00 AM,0.9288,2.16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86824,Yakima,USA,WA,10226,$10K - $30K,Jams and Jellies,Peanut Butter,Food,Plato Chunky Peanut Butter,Yakima,USA,WA,23,Mid-Size Grocery,6/30/1997 12:00:00 AM,1.9600,4.90,2
86825,Yakima,USA,WA,10226,$10K - $30K,Meat,Hot Dogs,Food,Cutting Edge Chicken Hot Dogs,Yakima,USA,WA,23,Mid-Size Grocery,6/30/1997 12:00:00 AM,1.4229,4.59,3
86826,Yakima,USA,WA,10226,$10K - $30K,Pizza,Pizza,Food,PigTail Frozen Pepperoni Pizza,Yakima,USA,WA,23,Mid-Size Grocery,3/1/1997 12:00:00 AM,1.0752,3.36,2
86827,Yakima,USA,WA,10226,$10K - $30K,Vegetables,Fresh Vegetables,Food,Ebony Garlic,Yakima,USA,WA,23,Mid-Size Grocery,2/11/1997 12:00:00 AM,4.3680,10.40,4


In [33]:
print(raw_data.columns)

Index(['C. City', 'C. Country', 'C. State', 'Customer', 'Yearly Income',
       'Category', 'Subcategory', 'Family', 'Product', 'S. City', 'S. Country',
       'S. State', 'Store', 'Type', 'Sales Date', 'Store Cost', 'Store Sales',
       'Unit Sales'],
      dtype='object')


In [45]:
dt_customer = raw_data[['C. City', 'C. Country', 'C. State', 'Customer', 'Yearly Income']].drop_duplicates()
dt_customer

,C. City,C. Country,C. State,Customer,Yearly Income
0,Albany,USA,OR,117,$90K - $110K
14,Albany,USA,OR,188,$30K - $50K
43,Albany,USA,OR,274,$30K - $50K
63,Albany,USA,OR,304,$90K - $110K
80,Albany,USA,OR,422,$10K - $30K
...,...,...,...,...,...
86702,Yakima,USA,WA,9967,$110K - $130K
86733,Yakima,USA,WA,10068,$50K - $70K
86767,Yakima,USA,WA,10073,$50K - $70K
86787,Yakima,USA,WA,10225,$30K - $50K


In [46]:
dt_product = raw_data[['Category', 'Subcategory', 'Family', 'Product']].drop_duplicates()
dt_product

,Category,Subcategory,Family,Product
0,Baking Goods,Cooking Oil,Food,Super Corn Oil
1,Baking Goods,Sauces,Food,Landslide Tomato Sauce
2,Baking Goods,Sugar,Food,Landslide Brown Sugar
3,Bathroom Products,Conditioner,Non-Consumable,Consolidated Silky Smooth Hair Conditioner
4,Beer and Wine,Wine,Drink,Pearl Merlot Wine
...,...,...,...,...
9140,Jams and Jellies,Jam,Food,Super Strawberry Jam
9952,Vegetables,Fresh Vegetables,Food,High Top Broccoli
10119,Fruit,Fresh Fruit,Food,High Top Lemons
10575,Eggs,Eggs,Food,Blue Medal Large Eggs


In [47]:
dt_store = raw_data[['S. City', 'S. Country',
       'S. State', 'Store', 'Type']].drop_duplicates()
dt_store

,S. City,S. Country,S. State,Store,Type
0,Salem,USA,OR,13,Deluxe Supermarket
2183,Los Angeles,USA,CA,7,Supermarket
2194,Beverly Hills,USA,CA,6,Gourmet Supermarket
3024,Bellingham,USA,WA,2,Small Grocery
4274,Seattle,USA,WA,15,Supermarket
5109,Portland,USA,OR,11,Supermarket
8015,San Francisco,USA,CA,14,Small Grocery
9008,Bremerton,USA,WA,3,Supermarket
14933,San Diego,USA,CA,24,Supermarket
41581,Tacoma,USA,WA,17,Deluxe Supermarket


In [49]:
dt_date = raw_data[['Sales Date']].drop_duplicates()
dt_date

,Sales Date
0,1/21/1997 12:00:00 AM
6,1/11/1997 12:00:00 AM
14,8/24/1997 12:00:00 AM
15,9/10/1997 12:00:00 AM
16,2/19/1997 12:00:00 AM
...,...
77990,3/7/1997 12:00:00 AM
83177,3/26/1997 12:00:00 AM
83250,9/16/1997 12:00:00 AM
83352,2/24/1997 12:00:00 AM


In [56]:
ft_sales = raw_data[['Sales Date', 'Store', 'Unit Sales', 'Product',
                     'Customer', 'Store Sales', 'Store Cost', 'Unit Sales']].drop_duplicates()
ft_sales

,Sales Date,Store,Unit Sales,Product,Customer,Store Sales,Store Cost,Unit Sales
0,1/21/1997 12:00:00 AM,13,3,Super Corn Oil,117,5.43,2.1177,3
1,1/21/1997 12:00:00 AM,13,4,Landslide Tomato Sauce,117,5.00,1.6500,4
2,1/21/1997 12:00:00 AM,13,4,Landslide Brown Sugar,117,11.20,5.3760,4
3,1/21/1997 12:00:00 AM,13,2,Consolidated Silky Smooth Hair Conditioner,117,2.94,1.1466,2
4,1/21/1997 12:00:00 AM,13,4,Pearl Merlot Wine,117,2.16,0.9288,4
...,...,...,...,...,...,...,...,...
86824,6/30/1997 12:00:00 AM,23,2,Plato Chunky Peanut Butter,10226,4.90,1.9600,2
86825,6/30/1997 12:00:00 AM,23,3,Cutting Edge Chicken Hot Dogs,10226,4.59,1.4229,3
86826,3/1/1997 12:00:00 AM,23,2,PigTail Frozen Pepperoni Pizza,10226,3.36,1.0752,2
86827,2/11/1997 12:00:00 AM,23,4,Ebony Garlic,10226,10.40,4.3680,4


In [42]:
host = "database-lab.cepvkrc8ii6y.us-east-1.rds.amazonaws.com"
port = 5432
user = "postgres"
pwd = "bigdata2022"
db = "postgres"

from sqlalchemy import create_engine
s = 'postgresql://{}:{}@{}:{}/{}'.format(user, pwd, host, str(port), db)
engine = create_engine(s)

In [54]:
dt_customer.to_sql('customer', engine, index=False, if_exists='replace')
dt_product.to_sql('product', engine, index=False, if_exists='replace')
dt_store.to_sql('store', engine, index=False, if_exists='replace')
dt_date.to_sql('date', engine, index=False, if_exists='replace')

In [57]:
ft_sales.to_sql('sale', engine, index=False, if_exists='replace')

## Query the DWH with Tableau (Desktop)

- Import data from Amazon RDS into Tableau
- Join the tables
- Set `Sales Date` as a Datetime attribute both in the fact and dimension tables
- Build the dimensions and hierarchies in Tableau
- Complete the following exercises

### Exercise 1

1. Using a bar chart, display the `sum` of `Store Sales` for each `S. State`
     - Which state has the highest sales?
2. Drill down to view sales at the `S. City` level.
     - Are there cities with much lower sales than others?
3. How many stores (`Stores`) are there in each `S. State`? What about in each `S. City`?
     - Tip: Use `COUNTD` aggregation. Why?
4. Can low sales in some cities be imputed to the number of stores?
     - Display sales at the `S. City` level (as done above) and associate the color property with the number of distinct stores (`COUNTD(store)`)

### Exercise 2

5. Given the last visualization, associate the `S. Type` field with the color property.
    - What interesting pattern can you see?
6. Display the sales (`SUM`) for each `S. Type`.
    - What discrepancy is possible to notice with respect to the previous chart?
7. Associate the number of stores (`Store`) with the color property and the label property
    - What caused the discrepancy between the previous two visualizations?

### Exercise 3

8. Using a line chart, visualize the monthly sales trend
    - What pattern is present?
9. Divide the previous chart by `S. State` (one axis for each state)
    - Is the previous pattern present in each `State`?
    - Tip: by default the axes all have the same range: on any axis, right-click > `Edit Axis` > `Select Independent axis...`
10. Given the previous chart, visualize how much impact the various `Family`s have on total sales while maintaining the monthly trend visualization
    - What might be a good visualization?
    - Tip: associate each Family with a mark property and possibly change the mark type
    - Tip: you can change the mark type from the drop-down menu in the mark panel

## (Bonus) Plot the provenance graph

- Plot the provenance graph representing the previous transformations using the PROV standard

In [6]:
from prov.model import ProvDocument
from prov.dot import prov_to_dot
from IPython.display import Image

def plot():
    # Create a new provenance document
    d1 = ProvDocument()  # d1 is now an empty provenance document
    
    d1.add_namespace('s3', 'https://s3.console.aws.amazon.com/s3')
    d1.add_namespace('rds', 'https://us-east-1.console.aws.amazon.com/rds')
    d1.add_namespace('unibo', 'https://www.unibo.it')
    d1.add_namespace('pd', 'https://pandas.pydata.org/')
    
    # TODO

    # visualize the graph
    dot = prov_to_dot(d1)
    dot.write_png('prov.png')
    return Image('prov.png')

In [7]:
plot()